In [126]:
import tensorflow as tf
import pickle
import numpy as np
import time
from load_cifar import *

In [116]:
tf.reset_default_graph()

<h1>Hyperparameter</h1>

In [117]:
# Hyperparameters
mu = 0
sigma = 0.1
epoch = 10
batch_size = 64
lr = 0.001

In [118]:
tf.reset_default_graph()

<h1>Placeholder</h1>

In [119]:
X = tf.placeholder(tf.float32,[None,32,32,3],name='X')
Y = tf.placeholder(tf.int32,[None,10],name='Y')

<h1>LeNet-5</h1>

In [120]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    conv1_w = tf.Variable(tf.truncated_normal(shape = [5,5,3,6],mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1 = tf.nn.conv2d(x,conv1_w, strides = [1,1,1,1], padding = 'VALID') + conv1_b 
    # Activation.
    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = 28x28x6. Output = 14x14x6.
    pool_1 = tf.nn.max_pool(conv1,ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    # Layer 2: Convolutional. Output = 10x10x16.
    conv2_w = tf.Variable(tf.truncated_normal(shape = [5,5,6,16], mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2 = tf.nn.conv2d(pool_1, conv2_w, strides = [1,1,1,1], padding = 'VALID') + conv2_b
    # Activation.
    conv2 = tf.nn.relu(conv2)

    # Pooling. Input = 10x10x16. Output = 5x5x16.
    pool_2 = tf.nn.max_pool(conv2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID') 
    
    # Flatten. Input = 5x5x16. Output = 400.
    fc1 = flatten(pool_2)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_w = tf.Variable(tf.truncated_normal(shape = (400,120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1 = tf.matmul(fc1,fc1_w) + fc1_b
    
    # Activation.
    fc1 = tf.nn.relu(fc1)
    
    # Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_w = tf.Variable(tf.truncated_normal(shape = (120,84), mean = mu, stddev = sigma))
    fc2_b = tf.Variable(tf.zeros(84))
    fc2 = tf.matmul(fc1,fc2_w) + fc2_b
    # Activation.
    fc2 = tf.nn.tanh(fc2)
    
    dropout = tf.layers.dropout(fc2,rate=0.2,noise_shape=None,seed=None,training=False,name=None)
    
    # Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_w = tf.Variable(tf.truncated_normal(shape = (84,10), mean = mu , stddev = sigma))
    fc3_b = tf.Variable(tf.zeros(10))
    logits = tf.matmul(dropout, fc3_w) + fc3_b
    return logits

<h1>Cost and Optimization</h1>

In [121]:
logits = LeNet(X)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = Y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = lr)
training_operation = optimizer.minimize(loss_operation)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
num_correct = tf.reduce_sum(tf.cast(correct_prediction,tf.float32))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

<h1>Training, validation and testing</h1>
<h2>Train your model only 10 epochs.</h2>
<h2>1.Print out validation accuracy after each training epoch</h2>
<h2>2.Print out training time for each training epoch</h2>
<h2>3.Print out testing accuracy</h2>

In [122]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(1,epoch+1):
        #fetch batch
        start_time = time.time()
        for batch_id in range(1,6):
            # print('Training for Batch: \t' + str(batch_id))
            training_data = load_preprocessed_training_batch(batch_id, batch_size)
            for data, labels in training_data:
                f_dict = {X: features_reshape(data), Y: labels}
                sess.run(training_operation, feed_dict=f_dict)
        end_time = time.time()

        print('Epoch'+str(i))

        val_correct_sum = 0
        val_total = 0
        val_data = load_preprocessed_validation_batch(batch_size)
        for data, labels in val_data:
            feed_dict = {X: features_reshape(data), Y: labels}
            batch_correct_count = sess.run(num_correct, feed_dict)
            val_correct_sum += batch_correct_count
            
        val_acc = val_correct_sum / 5000
        print("validation accuracy= {:.3f}".format(val_acc))

        test_correct_sum = 0
        test_data = load_preprocessed_test_batch(batch_size)
        test_total = 0
        for data, labels in test_data:
            feed_dict = {X: features_reshape(data), Y: labels}
            batch_correct_count = sess.run(num_correct, feed_dict)
            test_correct_sum += batch_correct_count
        test_acc = test_correct_sum / 10000
        print("test accuracy= {:.3f}".format(test_acc))

        print("--- %s seconds ---" % (end_time - start_time))

            
    
    print("Training finished!")



Epoch1
validation accuracy= 0.488
test accuracy= 0.481
--- 5.304867744445801 seconds ---
Epoch2
validation accuracy= 0.536
test accuracy= 0.527
--- 5.181605339050293 seconds ---
Epoch3
validation accuracy= 0.562
test accuracy= 0.551
--- 5.175970077514648 seconds ---
Epoch4
validation accuracy= 0.583
test accuracy= 0.567
--- 5.160879373550415 seconds ---
Epoch5
validation accuracy= 0.598
test accuracy= 0.585
--- 5.154704332351685 seconds ---
Epoch6
validation accuracy= 0.602
test accuracy= 0.589
--- 5.150904178619385 seconds ---
Epoch7
validation accuracy= 0.605
test accuracy= 0.592
--- 5.167161226272583 seconds ---
Epoch8
validation accuracy= 0.604
test accuracy= 0.594
--- 5.172987222671509 seconds ---
Epoch9
validation accuracy= 0.606
test accuracy= 0.596
--- 5.174516201019287 seconds ---
Epoch10
validation accuracy= 0.612
test accuracy= 0.600
--- 5.1498122215271 seconds ---
Training finished!
